<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="../images/DLI_Header.png"></a></div>

**This is the Solution File for this notebook. Please try the [Challenge Version](../1-04_content_based_real_data.ipynb) first.**


# Content Based Filters with Real Data

To test our Content-based Filters Knowledge, we'll be using a subsection of the [Amazon Review Dataset](https://nijianmo.github.io/amazon/index.html), specifically the Electronics dataset. There are a few more challenges working with real data such as data cleanliness, memory contraints, and  how to verify the model is ready for real world testing.

## Objectives
This notebook demonstrates:
* How to split data to train and test a recommender model
  * [1. Exploring the Data](#1.-Exploring-the-Data)
  * [2. Building the Datasets](#2.-Building-the-Datasets)
* How to build a Content-based filter for large datasets
  * [3. In-Place Content-Based Filters](#3.-In-Place-Content-Based-Filters)
    * [3.1 Finding the Total Number of Points Given per User](#3.1-Finding-the-Total-Number-of-Points-Given-per-User)
    * [3.2 Finding the Total Number of Points Given per User per Category](#3.2-Finding-the-Total-Number-of-Points-Given-per-User-per-Category)
    * [3.3 Finding the User's Percentage Preference for Each Category](#3.3-Finding-the-User's-Percentage-Preference-for-Each-Category)
  * [4. Making a Prediction](#4.-Making-a-Prediction)
  * [5. Validation Metrics](#5.-Validation-Metrics)
  * [6. Wrap Up](#6.-Wrap-Up)

## 1. Exploring the Data

Our data is located in a csv file under `/data`. We've already merged the metadata with the reviews data so everything can be displayed in one DataFrame.

In [1]:
import cudf

ratings = cudf.read_csv("../data/raw_data.csv")

ratings.columns

Index(['reviewerID', 'asin', 'overall', 'unixReviewTime', 'brand',
       'category_0_0', 'category_0_1', 'category_0_2', 'category_0_3',
       'category_1_0', 'category_1_1', 'category_1_2', 'category_1_3',
       'salesRank_Electronics', 'salesRank_Camera', 'salesRank_Computers',
       'salesRank_CellPhones', 'salesRank_CellPhones_NA',
       'salesRank_Electronics_NA', 'salesRank_Camera_NA',
       'salesRank_Computers_NA', 'price_filled'],
      dtype='object')

Hmm, it looks like our categories are being stored as strings alongside the user-item ratings. Previously, we had a column for each category with a `1` if the item was that category, and `0` otherwise. Can we make that here? 

Let's start by getting a list of all the categories. We can [concatenate](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.reshape.concat) to stack all the category columns together, and then use [unique](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.series.Series.unique) to remove any duplicates.

In [2]:
category_columns = [
    'category_0_0', 'category_0_1', 'category_0_2', 'category_0_3',
    'category_1_0', 'category_1_1', 'category_1_2', 'category_1_3'
]

categories = cudf.Series(dtype="str")
for category in category_columns:
    categories = cudf.concat([categories, ratings[category]])
categories = categories.unique()

categories

0                       3D Glasses
1                      AC Adapters
2        AV Receivers & Amplifiers
3           Access-Control Keypads
4                      Accessories
                  ...             
581                       Xbox 360
582                       Xbox One
583                      Zip Discs
584                  eBook Readers
585    eBook Readers & Accessories
Length: 586, dtype: object

Wow! 586 categories! We could have a column for each category like before, but is that wise? How many rows does our data have?

In [3]:
count_ratings = len(ratings)

count_ratings

1689188

If we do create a column for each value, we'll have a billion cells worth of data. Okay, new strategy. We're going to  build our filter using only a few more extra columns with our dataset.

## 2. Building the Datasets

Before we get much further, we should split our dataset into training and validation datasets. One thing to consider when doing this for recommendation systems is that our goal is to encourage future interaction by the users. Instead of doing a random split, we'll use the last review left by the user for validation.

The `unixReviewTime` column enables us to do that. We will start by finding the largest (newest) review time and assigning it a marker, `valid`. However, the `unixReviewTime` only goes down to the day, so if a user does all of their reviews on the same day, then all of their reviews would end up in the validation set. We can add a tie-breaker, or `timeBreaker`, by adding a scaled version of the row index to the time. 

The `as_index` parameter for [groupby](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.groupby) will create a fresh numerical index as opposed to using the `reviewerID` as the index.

In [4]:
scaled_index = ratings.index / count_ratings
ratings["timeBreaker"] = scaled_index + ratings["unixReviewTime"]

In [5]:
valid_ratings = ratings[["reviewerID", "timeBreaker"]].groupby(['reviewerID'], as_index=False).max()
valid_ratings["valid"] = True

valid_ratings.head()

,reviewerID,timeBreaker,valid
0,A000715434M800HLCENK9,1.400458e+09,True
1,A00101847G3FJTWYGNQA,1.385770e+09,True
2,A00166281YWM98A3SVD55,1.370563e+09,True
3,A0046696382DWIPVIWO0K,1.402877e+09,True
4,A00472881KT6WR48K907X,1.402358e+09,True


Next, we [merge](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.merge) the validation marker back onto our full ratings dataset to make it easier to pull the training data.

In [6]:
data_split = ratings.merge(valid_ratings, how="left", on=['reviewerID', "timeBreaker"])

data_split.head()

,reviewerID,asin,overall,unixReviewTime,brand,category_0_0,category_0_1,category_0_2,category_0_3,category_1_0,...,salesRank_Camera,salesRank_Computers,salesRank_CellPhones,salesRank_CellPhones_NA,salesRank_Electronics_NA,salesRank_Camera_NA,salesRank_Computers_NA,price_filled,timeBreaker,valid
0,AEVNTIQFU2TQ6,B00004ZCB3,3.0,1398643200,Tiffen,Electronics,Camera & Photo,Accessories,Filters & Accessories,NA,...,18665.5,14369.5,254050.0,False,False,False,False,16.49,1.398643e+09,<NA>
1,A1XI6NT41B6E6X,B00004ZCB3,5.0,1386288000,Tiffen,Electronics,Camera & Photo,Accessories,Filters & Accessories,NA,...,18665.5,14369.5,254050.0,False,False,False,False,16.49,1.386288e+09,<NA>
2,A2PMIMM3U3THM7,B00004ZCB3,5.0,1241481600,Tiffen,Electronics,Camera & Photo,Accessories,Filters & Accessories,NA,...,18665.5,14369.5,254050.0,False,False,False,False,16.49,1.241482e+09,<NA>
3,A1E9NFMLMVY61H,B00004ZCB4,5.0,1374624000,Tiffen,Electronics,Camera & Photo,Accessories,Filters & Accessories,NA,...,18665.5,14369.5,254050.0,False,False,False,False,16.49,1.374624e+09,<NA>
4,AXABTEYS7A4A8,B00004ZCB4,5.0,1374710400,Tiffen,Electronics,Camera & Photo,Accessories,Filters & Accessories,NA,...,18665.5,14369.5,254050.0,False,False,False,False,16.49,1.374710e+09,<NA>


To keep things clean, we'll replace the `null`s in the `valid` column with `False`.

In [7]:
data_split['valid'].fillna(False, inplace = True)

We'll keep a simplified table of the user-item validation ratings to make it easier to check our work later.

In [8]:
clean_columns = ["reviewerID", "asin", "overall"]
train_overall = data_split.loc[~data_split['valid']][clean_columns]

train_overall.head()

,reviewerID,asin,overall
0,AEVNTIQFU2TQ6,B00004ZCB3,3.0
1,A1XI6NT41B6E6X,B00004ZCB3,5.0
2,A2PMIMM3U3THM7,B00004ZCB3,5.0
3,A1E9NFMLMVY61H,B00004ZCB4,5.0
4,AXABTEYS7A4A8,B00004ZCB4,5.0


In [9]:
valid_overall = data_split.loc[data_split['valid']][clean_columns]

valid_overall.head()

,reviewerID,asin,overall
27616,A3MICRVMOFR97Z,B000067V62,4.0
27648,A2HYCY9E1PJX4G,B00005LE74,4.0
27712,A242IFN64X6R4E,B000067VBM,4.0
27776,A2HNSWGBPZ2E3B,B00005RZYY,5.0
27840,A38986T7SECVKX,B00004SB92,5.0


While we're at it, let's save some of this work for future notebooks.

In [10]:
save_columns = clean_columns + ['valid']

data_split[save_columns].to_csv('../data/ratings.csv', index=False)

Here's the overall data efficiency strategy. We'll stack all the categories into one column, and then use `groupby` operations to get our user ratings for each category.

We'll again use the [concat](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.reshape.concat) function, but if we want all the categories to fall under the same column, we'll need to use the [rename](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.series.Series.rename) function to give each stack section the same category name.

A little more data cleanup... We'll also remove the `NA` category since it's an empty category, the `Electronics` category since every item has it, and any duplicate categories.

In [11]:
column_base = ["reviewerID", "asin", "overall", "valid"]

ratings_stack = cudf.DataFrame()
for category in category_columns:
    stack_section = data_split[column_base + [category]]
    stack_section = stack_section.rename(columns={category: "category"})
    ratings_stack = cudf.concat([ratings_stack, stack_section])

# Remove null categories and duplicates
ratings_stack = ratings_stack.loc[ratings_stack['category'] != "NA"]
ratings_stack = ratings_stack.loc[ratings_stack['category'] != "Electronics"]
ratings_stack = ratings_stack.drop_duplicates()

ratings_stack.head()

,reviewerID,asin,overall,valid,category
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,False,Accessories & Supplies
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,False,Office Electronics Accessories
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,False,Projection Screens
460368,A000715434M800HLCENK9,B001EHAI6Y,5.0,False,MP3 Player Accessories
460368,A000715434M800HLCENK9,B001EHAI6Y,5.0,False,MP3 Players & Accessories


Finally, we can use our `valid` marker and [loc](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.loc) to split our datasets.

In [12]:
train_data = ratings_stack.loc[~ratings_stack['valid']].drop("valid", axis=1)

train_data.head()

,reviewerID,asin,overall,category
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,Accessories & Supplies
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,Office Electronics Accessories
330955,A000715434M800HLCENK9,B000UYYZ0M,1.0,Projection Screens
460368,A000715434M800HLCENK9,B001EHAI6Y,5.0,MP3 Player Accessories
460368,A000715434M800HLCENK9,B001EHAI6Y,5.0,MP3 Players & Accessories


In [13]:
valid_data = ratings_stack.loc[ratings_stack['valid']].drop("valid", axis=1)

valid_data.head()

,reviewerID,asin,overall,category
1682551,A000715434M800HLCENK9,B00HMZG3YS,5.0,Bags & Cases
1682551,A000715434M800HLCENK9,B00HMZG3YS,5.0,Camera & Photo
1682551,A000715434M800HLCENK9,B00HMZG3YS,5.0,Camera Cases
1537728,A00101847G3FJTWYGNQA,B00B19L8LO,4.0,Computer Cases
1537728,A00101847G3FJTWYGNQA,B00B19L8LO,4.0,Computer Components


## 3. In-Place Content-Based Filters


Now that we have our user (`reviewerID`), item (`asin`), rating (`overall`), and category all lined up, we have the necessary ingredients to make a content-based filter.

The following is a puzzle. Please replace the `FIXME`s for each code block in the section. Each `FIXME` is a single string or method. Use the <a href="1-03_content_based_intro.ipynb">previous lab</a> as a hint, and if more help is needed, take a look at the solutions folder.

### 3.1 Finding the Total Number of Points Given per User


In [14]:
user_total = (
    train_data[["reviewerID", "overall"]].groupby(["reviewerID"], as_index=False).sum()
)

user_total.head()

,reviewerID,overall
0,A000715434M800HLCENK9,33.0
1,A00101847G3FJTWYGNQA,67.0
2,A00166281YWM98A3SVD55,58.0
3,A0046696382DWIPVIWO0K,43.0
4,A00472881KT6WR48K907X,84.0


### 3.2 Finding the Total Number of Points Given per User per Category

In [15]:
category_total = (
    train_data[["reviewerID", "overall", "category"]]
    .groupby(["reviewerID", "category"], as_index=False)
    .sum()
)

category_total.head()

,reviewerID,category,overall
0,A000715434M800HLCENK9,Accessories & Supplies,6.0
1,A000715434M800HLCENK9,Audio & Video Accessories,2.0
2,A000715434M800HLCENK9,Cables & Interconnects,2.0
3,A000715434M800HLCENK9,MP3 Player Accessories,5.0
4,A000715434M800HLCENK9,MP3 Players & Accessories,5.0


### 3.3 Finding the User's Percentage Preference for Each Category

In [16]:
category_total = category_total.merge(user_total, how="left", on='reviewerID')

category_total.head()

,reviewerID,category,overall_x,overall_y
0,A10Y058K7B96C6,Camera Cases,14.0,1465.0
1,A10Y058K7B96C6,Car & Vehicle Electronics,12.0,1465.0
2,A10Y058K7B96C6,Cases,5.0,1465.0
3,A10Y058K7B96C6,Cases & Sleeves,34.0,1465.0
4,A10Y058K7B96C6,Chargers & Adapters,12.0,1465.0


In [17]:
category_total["ratio"] = category_total["overall_x"] / category_total["overall_y"]

category_total.head()

,reviewerID,category,overall_x,overall_y,ratio
0,A10Y058K7B96C6,Camera Cases,14.0,1465.0,0.009556
1,A10Y058K7B96C6,Car & Vehicle Electronics,12.0,1465.0,0.008191
2,A10Y058K7B96C6,Cases,5.0,1465.0,0.003413
3,A10Y058K7B96C6,Cases & Sleeves,34.0,1465.0,0.023208
4,A10Y058K7B96C6,Chargers & Adapters,12.0,1465.0,0.008191


In [18]:
category_total = category_total.drop(["overall_x", "overall_y"], axis=1)

category_total.head()

,reviewerID,category,ratio
0,A10Y058K7B96C6,Camera Cases,0.009556
1,A10Y058K7B96C6,Car & Vehicle Electronics,0.008191
2,A10Y058K7B96C6,Cases,0.003413
3,A10Y058K7B96C6,Cases & Sleeves,0.023208
4,A10Y058K7B96C6,Chargers & Adapters,0.008191


## 4. Making a Prediction

Congrats on making the users' profiles. Let's use them to make a prediction! We'll start by [merging](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.merge) our predictions with our validation data. We'll use an `inner` merge to avoid including predictions outside our validation dataset.

We do not need the validation ratings just yet, but we do need the categories for each item in our validation set. This allows us to pull the relevant categories from the user profile to find how well their category tastes align with the categories of the item we're predicting for.

In [19]:
prediction = category_total.merge(
    valid_data, how="inner", on=["reviewerID", "category"]
)

prediction.head()

,reviewerID,category,ratio,asin,overall
0,A10VB212O1V4J3,Accessories,0.030534,B00COYOAYW,4.0
1,A128LJR0ZMS5W1,Camera & Photo,0.271605,B004GBIDEO,5.0
2,A12EVK82QUTSA6,Radar Detectors,0.005656,B00HK5B3PO,4.0
3,A10O1SW6BBO00O,Camera & Photo,0.248408,B00823EE8G,5.0
4,A13L24Q3OKM0VK,Camera & Photo,0.097701,B00G24Z98G,4.0


In [20]:
prediction = prediction.drop(["category", "overall"], axis=1).groupby(['reviewerID', 'asin'], as_index=False).sum()

prediction.head()

,reviewerID,asin,ratio
0,A00101847G3FJTWYGNQA,B00B19L8LO,0.432836
1,A00166281YWM98A3SVD55,B007B5S8BU,0.344828
2,A0046696382DWIPVIWO0K,B00HMXIKCS,0.279070
3,A00472881KT6WR48K907X,B00GYL9KK0,0.059524
4,A00473363TJ8YSZ3YAGG9,B003NR57BY,0.395833


## 5. Validation Metrics

The best way to test a recommender model is to do [A/B testing](https://en.wikipedia.org/wiki/A/B_testing), as the goal is to maximize user interactivity with our systems. However, we do not want to risk turning tested users away with a bad model, so we can sanity check our model against other metrics beforehand.

Here, we'll calculate [Root Mean Squared Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) with our validation set. Let's merge our predicted ratings with the validation data's true rating.

In [21]:
prediction = prediction.merge(valid_overall, how="inner", on=['reviewerID', 'asin'])

prediction.head()

,reviewerID,asin,ratio,overall
0,A23P9EWNJWUP1G,B001441L9A,0.073529,5.0
1,A3H4T0GVLMG822,B001441L9A,0.352941,5.0
2,APL0945T2OTV,B001441L9A,0.421053,5.0
3,A2VD05UT6B7RE6,B001441L9A,0.375000,5.0
4,A1SFMKZZ9M5521,B001445TJS,0.176796,4.0


As in our example problem [in the previous notebook](1-03_content_based_intro.ipynb), we'll need to scale up our predictions.

Please replace the `FIXME`s so that we have predicted ratings with the correct scale.

In [22]:
min_rating = 1
max_rating = 5
prediction["predicted_rating"] = prediction["ratio"] * (max_rating - min_rating) + min_rating

prediction.head()

,reviewerID,asin,ratio,overall,predicted_rating
0,A23P9EWNJWUP1G,B001441L9A,0.073529,5.0,1.294118
1,A3H4T0GVLMG822,B001441L9A,0.352941,5.0,2.411765
2,APL0945T2OTV,B001441L9A,0.421053,5.0,2.684211
3,A2VD05UT6B7RE6,B001441L9A,0.375000,5.0,2.500000
4,A1SFMKZZ9M5521,B001445TJS,0.176796,4.0,1.707182


Next, we'll calculate the squared error by subtracting the true value from the predicted value and squaring the result.

In [23]:
prediction["squared_error"] = (
    prediction["predicted_rating"] - prediction["overall"]
) ** 2

prediction.head()

,reviewerID,asin,ratio,overall,predicted_rating,squared_error
0,A23P9EWNJWUP1G,B001441L9A,0.073529,5.0,1.294118,13.733564
1,A3H4T0GVLMG822,B001441L9A,0.352941,5.0,2.411765,6.698962
2,APL0945T2OTV,B001441L9A,0.421053,5.0,2.684211,5.362881
3,A2VD05UT6B7RE6,B001441L9A,0.375000,5.0,2.500000,6.250000
4,A1SFMKZZ9M5521,B001445TJS,0.176796,4.0,1.707182,5.257013


Finally, we can find the Root Mean Squared Error (RMSE) by [averaging](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.mean) the `squared_error`s and taking the square root.

In [24]:
prediction["squared_error"].mean() ** .5

2.6019044951648502

So, roughly on average, the predictions are off by about 2 points. Considering the scale of ratings goes from 1 - 5, that's a pretty big error.

However, there are some things to note.
* Predicted ratings are relative to other items. Take the example where a user would rate item A 5 and item B 1. It doesn't matter so much that our system predicts {A 4, B 3} or {A 3, B 2.5}. In both cases, item A is recommended to the user over B.
* In the next notebook, we'll go over collaborative filtering, which can produce better predictions with more data. There are some instances where content-based filtering can make predictions where collaborative filtering cannot. Try to keep track of the differences while going through the next notebook.

## 6. Wrap Up

Congratulations on making it this far! Please run the cell below to shut down the kernel before moving on to the <a href="1-05_als.ipynb">next notebook</a>.

In [25]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="../images/DLI_Header.png"></a></div>